## 490C Final Project
#### Dylan Toothaker & Ravi Patel

In [1]:
# Set-up
import time
from bs4 import BeautifulSoup
from lxml import html
import matplotlib
import urllib
from datascience import *
import os
import re
import random
import numpy as np


from collections import Counter
from scipy.stats import entropy
import os
import re
import numpy as np
from collections import Counter
from scipy.stats import entropy

import re
import numpy as np
from collections import Counter

from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style='white', font_scale=1.3) # Set the style for the plots

# Modules from scikit-learn
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.naive_bayes import MultinomialNB

# Packages for plotting
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style='white', font_scale=1.3) # Set the style for the plots
# scikit-learn modules & functions
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering


### Load Data

This function runs through our directory, and adds each genre dataset to a master dataset. After, we write the master as a single .csv file. Now we can load in the data with a single read_csv call. 

load_genre_reviews() takes over 10 minutes to run. 

In [2]:
def load_genre_reviews():
    word_pattern = re.compile("\w[\w\-\']*\w|\w")
    genre_reviews = Table(['ID', 'Movie', 'Genre', 'Age Rating', 'Star Rating', 'Review URL'])
    id_count = 0
    
    review_list_o = []
    
    for file in os.listdir("genres"):
        genre = file
        filepath = 'genres/{}'.format(file)
        
        with open(filepath, encoding='utf-8') as reader:
            first_line = True
            print(genre)
            for line in reader:
                if (first_line):
                    first_line = False
                    continue
                
                id_count = id_count+1
                split_line = line.split('","')
                cur_name = split_line[0]
                cur_genre = genre[:-4]
                cur_age_rating = split_line[2]
                cur_star_rating = split_line[6]
                cur_review_url = (split_line[9][:-3])
                review_list_i = []
                
                
                url_contents = urllib.request.urlopen(cur_review_url).read()
                soup = BeautifulSoup(url_contents, "html")
                soup = soup.find_all("div", {'id', 'text show-more__control'})
                
                for review in soup:
                    review_list_i.append(str(review)[37:-6])
                
                review_list_o.append(review_list_i)
                
                
                genre_reviews.append([id_count, cur_name, cur_genre, cur_age_rating, cur_star_rating, cur_review_url])

                
    return genre_reviews, review_list_o

In [3]:
master, row = load_genre_reviews()
master = master.with_column('Reviews', row)

In [4]:
master.to_csv('master.csv')

### Show Data

In [5]:
master = Table.read_table('master.csv')
master.show()

ID,Movie,Genre,Age Rating,Star Rating,Review URL,Reviews
1,"""The Dark Knight",Action,PG-13,9,https://www.imdb.com/title/tt0468569/reviews/_ajax?ref_= ...,"['Confidently directed, dark, brooding, and packed with ..."
2,"""Inception",Action,PG-13,8.8,https://www.imdb.com/title/tt1375666/reviews/_ajax?ref_= ...,"['My 3rd time watching this movie! Yet, it still stunned ..."
3,"""The Matrix",Action,R,8.7,https://www.imdb.com/title/tt0133093/reviews/_ajax?ref_= ...,"['When this came out, I was living with a roommate. He w ..."
4,"""The Lord of the Rings: The Fellowship of the Ring",Action,PG-13,8.8,https://www.imdb.com/title/tt0120737/reviews/_ajax?ref_= ...,['Director Peter Jackson and wife Fran Walsh have succes ...
5,"""The Dark Knight Rises",Action,PG-13,8.4,https://www.imdb.com/title/tt1345836/reviews/_ajax?ref_= ...,"[""Christopher Nolan's epic trilogy concludes in glorious ..."
6,"""Gladiator",Action,R,8.5,https://www.imdb.com/title/tt0172495/reviews/_ajax?ref_= ...,"['Ridley Scott\'s Gladiator is not a perfect film, I wou ..."
7,"""Batman Begins",Action,PG-13,8.2,https://www.imdb.com/title/tt0372784/reviews/_ajax?ref_= ...,"[""It sickened me in the past to see the Batman movie fra ..."
8,"""The Avengers",Action,PG-13,8,https://www.imdb.com/title/tt0848228/reviews/_ajax?ref_= ...,"[""Up until this point, nothing had been done like this i ..."
9,"""Star Wars: Episode IV - A New Hope",Action,PG,8.6,https://www.imdb.com/title/tt0076759/reviews/_ajax?ref_= ...,['What made this the hugely successful triumph it was? W ...
10,"""Star Wars: Episode V - The Empire Strikes Back",Action,PG,8.7,https://www.imdb.com/title/tt0080684/reviews/_ajax?ref_= ...,"[""Star Wars: Episode V - The Empire Strikes Back (1980) ..."


#### Number of Movies

In [6]:
print('Total Movies: ' + str(len(master.column('Movie'))))

Total Movies: 1300


In [7]:
print('Unique Movies: ' +  str(len(np.unique(master.column('Movie')))))

Unique Movies: 960


#### Average Rating

In [8]:
print('Average Rating: ' + str(np.mean(master.column('Star Rating').astype(float))))

Average Rating: 7.451615384615385


#### Total Reviews

In [9]:
count = 0
for i in master.column('Movie'):
    for j in i:
        count = count + 1
print('Total Reviews: ' + str(count))

Total Reviews: 21551


### Tokenization

In [10]:
def tokenize_review(text):
    return token_pattern.findall(str(text).lower())

In [11]:
def tokenize_movie(arr_text):
    result = []
    for text in arr_text:
        result = np.append(result, tokenize_review(text))

In [12]:
token_pattern = re.compile("\w[\w\-]*\w")
tokens = tokenize_review(master.column('Reviews'))

print(tokens)

['confidently', 'directed', 'dark', 'brooding', 'and', 'packed', 'with', 'impressive', 'action', 'sequences', 'and', 'complex', 'story', 'the', 'dark', 'knight', 'includes', 'career-defining', 'turn', 'from', 'heath', 'ledger', 'as', 'well', 'as', 'other', 'oscar', 'worthy', 'performances', 'tdk', 'remains', 'not', 'only', 'the', 'best', 'batman', 'movie', 'but', 'comic', 'book', 'movie', 'ever', 'created', 'got', 'to', 'see', 'the', 'dark', 'knight', 'on', 'wednesday', 'night', 'the', 'reason', 'though', 'why', 'writing', 'this', 'movie', 'comment', 'this', 'late', 'is', 'because', 'didn', 'wanna', 'just', 'jump', 'and', 'say', 'this', 'movie', 'was', 'awesome', 'wanted', 'to', 'think', 'it', 'through', 'still', 'today', 'can', 'stop', 'thinking', 'about', 'this', 'movie', 'the', 'dark', 'knight', 'lives', 'up', 'to', 'it', 'hype', 'and', 'goes', 'beyond', 'it', 'this', 'is', 'the', 'batman', 'movie', 'that', 'goes', 'where', 'no', 'other', 'batman', 'movie', 'has', 'gone', 'before', 

We are now at the point we have created a master dataset with all our genres. We also have tokenized all the reviews. 
Now we are creating multiple .txt files, each with the reviews for their respective genre. 

In [13]:
with open ("action_t.txt", 'w') as f:
    f.write(' '.join(tokenize_review(master.column('Reviews')[0:100])))
with open ("adventure_t.txt", 'w') as f:
    f.write(' '.join(tokenize_review(master.column('Reviews')[100:200])))
with open ("animation_t.txt", 'w') as f:
    f.write(' '.join(tokenize_review(master.column('Reviews')[200:300])))
with open ("biography_t.txt", 'w') as f:
    f.write(' '.join(tokenize_review(master.column('Reviews')[300:400])))
with open ("comedy_t.txt", 'w') as f:
    f.write(' '.join(tokenize_review(master.column('Reviews')[400:500])))
with open ("drama_t.txt", 'w') as f:
    f.write(' '.join(tokenize_review(master.column('Reviews')[500:600])))
with open ("fantasy_t.txt", 'w') as f:
    f.write(' '.join(tokenize_review(master.column('Reviews')[600:700])))
with open ("horror_t.txt", 'w') as f:
    f.write(' '.join(tokenize_review(master.column('Reviews')[700:800])))
with open ("music_t.txt", 'w') as f:
    f.write(' '.join(tokenize_review(master.column('Reviews')[800:900])))
with open ("mystery_t.txt", 'w') as f:
    f.write(' '.join(tokenize_review(master.column('Reviews')[900:1000])))
with open ("sport_t", 'w') as f:
    f.write(' '.join(tokenize_review(master.column('Reviews')[1000:1100])))
with open ("thriller_t.txt", 'w') as f:
    f.write(' '.join(tokenize_review(master.column('Reviews')[1000:1100])))
with open ("war_t.txt", 'w') as f:
    f.write(' '.join(tokenize_review(master.column('Reviews')[1100:1200])))

Now we create different counters for each genre so we can look at them individually. 

In [14]:
genres = ['action', "adventure", "animation", "biography", "comedy", "drama",
         "fantasy", "horror", "music", "mystery", "thriller", "war"]
# This counter will store the total frequency of each word type
all_counts = Counter()
genre_counts = {genre: Counter() for genre in genres}
movie_counts = Counter()

# A list of play-level data structures, one for each play in the collection
movie_data = []
movie_genres = []

In [15]:
word_pattern = re.compile("\w[\w\-\']*\w|\w")
genre_directories = {
    "action": "genres/action",
    "adventure": "genres/adventure",
    "animation": "genres/animation",
    "biography": "genres/biography",
    "comedy": "genres/comedy",
    "drama": "genres/drama",
    "fantasy": "genres/fantasy",
    "horror": "genres/horror",
    "music": "genres/music",
    "mystery": "genres/mystery",
    "thriller": "genres/thriller",
    "war": "genres/war",
}

for genre, genre_directory in genre_directories.items():
    for filename in os.listdir(genre_directory):
        # Only read .txt files
        if (filename.endswith('.txt')):
            # Initialize play-specific data structures
            play_counter = Counter()
            for row in master.rows:
                movie_review.append({
                    'genre': row[2],
                    'movie': row[1],
                    'reviews': row[6],
                    'counts': movie_counts.update(row)
                })
            movie_genres.append(genre)
            # Open and tokenize file
            filepath = '{}/{}'.format(genre_directory, filename)
            with open(filepath, encoding='utf-8') as reader: ## What encoding are the files in?
                for line in reader:
                    line = line.strip()
                    # Skip empty and non-speaking lines
                    if line and not line.startswith('<'):
                        line = line.lower()
                        tokens = word_pattern.findall(line)
                        play_counter.update(tokens)
                 
            
                movie_counter[row[1]].update(play_counter)
                        
            # Update genre and overall counters
            genre_counts[genre].update(play_counter)
            all_counts += play_counter

NameError: name 'movie_review' is not defined

In [ ]:
collection_length = sum([all_counts[term] for term in tokens])
genre_lengths = {genre: sum([genre_counts[genre][term] for term in tokens]) for genre in genres}

### Feature Analysis

In [ ]:
def get_contingency_table(term, genre):
    table = np.zeros((2,2))
    genre_length = genre_lengths[genre]
    genre_counter = genre_counts[genre]
    table[0, 0] = genre_counter[term]
    table[1, 0] = all_counts[term] - genre_counter[term]
    table[0, 1] = genre_length - genre_counter[term]
    table[1, 1] = (collection_length - genre_length) - (all_counts[term] - genre_counter[term])
    return table

def dunning_gscore(table, signed=False):
    rows = table.sum(axis=1)
    cols = table.sum(axis=0)
    score = 2 * table.sum() * (entropy(rows) + entropy(cols) - entropy(table.flatten()))
    if signed:
        assert(table.shape == (2,2))
        if table[0,0]/table[:, 0].sum() < table[0,1]/table[:, 1].sum():
            score *= -1
    return score

In [ ]:
table = get_contingency_table('funny', 'comedy')
dunning_gscore(table)

In [ ]:
def print_extremes(words, scores, k, is_top):
    k_pos = None
    if is_top:
        k_pos = np.argsort(scores)[::-1][:k]
    else:
        k_pos = np.argsort(scores)[:k]
    for i, pos in enumerate(k_pos):
        print("{}. {}: {}".format(i+1, words[pos], scores[pos]))

In [ ]:
print("Least distinctive words for the Comedy genre:")
print_extremes(list(genre_counts['comedy'].keys()), list(genre_counts['comedy'].values()), 10, False)

print("Least distinctive words for the Horror genre:")
print_extremes(list(genre_counts['horror'].keys()), list(genre_counts['horror'].values()), 10, False)

#### Most Characteristic words for Horror genre

In [ ]:
scores = []
for term in list(genre_counts['horror']):
    score = None
    
    term_table = get_contingency_table(term, 'horror')
    
    score = dunning_gscore(term_table, True)
    
    scores.append(score)
    
print("Most characteristic words for the Horror genre:")
print_extremes(list(genre_counts['horror'].keys()), scores, 20, True)

#### Most Characteristic words for Comedy genre:

In [ ]:
scores = []
for term in list(genre_counts['comedy']):
    score = None
    
    term_table = get_contingency_table(term, 'comedy')
    
    score = dunning_gscore(term_table, True)
    
    scores.append(score)
    
print("Most characteristic words for the Comedy genre:")
print_extremes(list(genre_counts['comedy'].keys()), scores, 20, True)


In [ ]:
scores = []
for term in list(genre_counts['sport']):
    score = None
    
    term_table = get_contingency_table(term, 'sport')
    
    score = dunning_gscore(term_table, True)
    
    scores.append(score)
    
print("Most characteristic words for the Sport genre:")
print_extremes(list(genre_counts['sport'].keys()), scores, 20, True)

In [ ]:
def contingency_table(count1, total1, count2, total2):
    table = np.zeros((2,2))
    table[0] = count1, count2
    table[1] = total1-count1, total2-count2
    return table

def dunning_gscore(table):
    rows = table.sum(axis=1)
    cols = table.sum(axis=0)
    score = 2 * table.sum() * (entropy(rows) + entropy(cols) - entropy(table.flatten()))
    return score

In [ ]:
def most_distinctive(counter_a, counter_b, n_words):
    len_a = sum(counter_a.values())
    len_b = sum(counter_b.values())
    
    vocabulary = set(counter_a.keys()) & set(counter_b.keys())
    
    score_info = []
    for word in vocabulary:
        count_a = counter_a[word]
        count_b = counter_b[word]
        table = contingency_table(count_a, len_a, count_b, len_b)
        score = dunning_gscore(table)
        score_info.append((score, word, count_a, count_b))
        
    score_info.sort(reverse=True)
    for i in range(n_words):
        item = score_info[i]
        print("{}. {:.1f} | {}: {} {}".format(i+1, item[0], item[1], item[2], item[3]))

In [ ]:
print('Rank Score| Term: Horror Comedy')
print('_______________________________')
most_distinctive(genre_counts['horror'], genre_counts['comedy'], 25)

In [ ]:
scores = []
N = sum(genre_counts['comedy'].values())
M = sum(genre_counts['horror'].values())
for term in tokens:
    score = None
    
    term_table = contingency_table(genre_counts['comedy'][term], N, genre_counts['horror'][term], M)
    score = dunning_gscore(term_table)
    
    scores.append(score)

In [ ]:
# Print the top k word-score pairs
def print_extremes(words, scores, k, is_top):
    k_pos = None
    if is_top:
        k_pos = np.argsort(scores)[::-1][:k]
    else:
        k_pos = np.argsort(scores)[:-k]
    temp = []
    count = 1
    for i, pos in enumerate(k_pos):
        if (([words[pos], scores[pos]] in temp) or (scores[pos] == 0.0)):
            continue
        temp.append([words[pos], scores[pos]])
        count = count + 1
        print("{}. {}: {}".format(count-1, words[pos], scores[pos]))
        if count == 21:
            break

In [ ]:
print("Most distinctive words in reviews of the horror genre and comedy genre:")
print_extremes(tokens, scores, 1000, True)

In [ ]:
print("Least distinctive words in reviews of the horror genre and comedy genre:")
print_extremes(tokens, scores, 1000, False)

### Classification

In [ ]:
def build_vectors(vocab):
    vectors = []
    for struct in movie_data:
        counts = struct['counts']
        vector = np.array([counts[w] for w in vocab])
        vectors.append(vector)
    return vectors

play_vectors = build_vectors(tokens)

In [ ]:
def classify_genre(vectors, smoothing=1, print_results=True):
    model = MultinomialNB(alpha=smoothing, fit_prior=False)
    cross_validator = LeaveOneOut()
    scores = cross_val_score(model, vectors, movie_genres, cv=cross_validator)
    print(scores)
    # Compute overall accuracy
    overall_accuracy = 100*sum(scores)/len(scores)
    if print_results:
        print("Overall Accuracy: {:.1f}%\n".format(100*sum(scores)/len(scores)))
        # Compute genre-level acuracies
        genre_scores = {g:[] for g in genres}
        for i, genre in enumerate(movie_genres):
            genre_scores[genre].append(scores[i])
        for g in genres:
            genre_name = g.title()
            print(genre_scores[g])
            print("{} Accuracy: {:.1f}%".format(genre_name,
                  100*(sum(genre_scores[g])/len(genre_scores[g]))))
    return overall_accuracy

In [ ]:
_ = classify_genre(play_vectors)

In [ ]:
def classify_genre(genre, vectors, smoothing=1):
    # Construct binary labels
    binary_labels = []
    for g in movie_genres:
        if g == genre:
            binary_labels.append(g)
        else:
            binary_labels.append("not_{}".format(genre))
    # Train binary classifier
    model = MultinomialNB(alpha=smoothing, fit_prior=False)
    cross_validator = LeaveOneOut()
    scores = cross_val_score(model, vectors, binary_labels, cv=cross_validator)
    # Compute overall accuracy
    print("{} - Accuracy: {:.1f}%".format(genre, 100*sum(scores)/len(scores)))

In [ ]:
for genre in genres:
    classify_genre(genre, play_vectors)

In [ ]:
def get_word_prob(word, genre):
    return genre_counts[genre][word] / sum(genre_counts[genre].values())

In [ ]:
genre_prob = Table()
genre_prob = genre_prob.with_column('GENRE', ['action', "adventure", "animation", "biography", "comedy", "drama",
         "fantasy", "horror", "music", "mystery", "thriller", "war"])
genres = ['action', "adventure", "animation", "biography", "comedy", "drama",
         "fantasy", "horror", "music", "mystery", "thriller", "war"]
genres2 = ['action', "adventure", "animation", "biography", "comedy", "drama",
         "fantasy", "horror", "music", "mystery", "thriller", "war"]

count1 = 1
count2 = 1
for genre in genres:
    row = []
    for genre2 in genres2:
        row.append(get_word_prob(genre,genre2))
        count1 = count1 + 1
    genre_prob = genre_prob.with_column(genre, row)
    count2 = count2 + 1
    
genre_prob.show()

### Comparison
#### Burrow's Delta / Cosine Similarity

In [ ]:
# A list of novel-level structures (dictionaries)
movie_data = []

# Load in data
movie_counter = Counter()
for row in master.rows:
    movie_data.append({
            'genre': row[2],
            'movie': row[1],
            'reviews': row[6],
            'counts': Counter(tokens)
        })
movie_counter.update(set(tokens))

In [ ]:
vocabulary = {}
for word in tokens:
    if word in vocabulary:
        vocabulary[word] = vocabulary[word] + 1
    else:
        vocabulary[word] = 1

In [ ]:
vocabulary2 = []
for word in vocabulary:
    if vocabulary[word] > 1:
        vocabulary2.append(word)
vocabulary2.sort()
print('Old Vocab Size: {}'.format(len(vocabulary)))
print('New Vocab Size: {}'.format(len(vocabulary2)))

In [ ]:
for struct in movie_data:
    vector = [struct['counts'][w] for w in vocabulary2]
    struct['vector'] = np.array(vector)

In [ ]:
def cosine(vector_a, vector_b):
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)
    return vector_a.dot(vector_b) / (norm_a * norm_b)

In [ ]:
similarities = []
for i in range(len(movie_data)):
    vec_i = movie_data[i]['vector']
    for j in range(i+1, len(movie_data)):
        vec_j = movie_data[j]['vector']
        similarities.append(cosine(vec_i, vec_j))
plt.figure(figsize=(8,3))
_ = sns.histplot(similarities, stat='count', bins=20, color='purple')
_ = plt.xlabel('Cosine Similarities')

In [ ]:
def jaccard(vector_a, vector_b):
    # Construct sets of the non-zero entries of each vector
    set_a = set(np.flatnonzero(vector_a)) 
    set_b = set(np.flatnonzero(vector_b))
    # Compute the intersection and union of the two sets
    overlap = set_a & set_b
    overall = set_a | set_b
    return len(overlap) / len(overall)

In [ ]:
similarities = []
for i in range(len(movie_data)):
    vec_i = movie_data[i]['vector']
    for j in range(i+1, len(movie_data)):
        vec_j = movie_data[j]['vector']
        similarities.append(jaccard(vec_i, vec_j))
plt.figure(figsize=(8,3))
_ = sns.histplot(similarities, stat='count', bins=20, color='purple')
_ = plt.xlabel('Jaccard Similarity')

### Clustering

In [ ]:
movie_review = []
movie_counter = Counter()
for row in master.rows:
    movie_review.append({
            'genre': row[2],
            'movie': row[1],
            'reviews': row[6],
            'counts': movie_counter
        })

In [ ]:
vocabulary3 = []
for word_type, doc_count in movie_counter.most_common():
    if doc_count > 1:
        vocabulary3.append(word_type)
print('Max Vocab Size: {}'.format(len(all_counts.keys())))
print('Current Vocab Size: {}'.format(len(vocabulary3)))

In [ ]:
# print(movie_data)

In [ ]:
##################################

movie_vectors = []
for struct in movie_data:
    counts = struct['counts']
    vector = np.array([counts[w] for w in vocabulary3])
    # Normalize vector's by their length
    vector = vector/vector.sum()
    movie_vectors.append(vector)

In [ ]:
movie_genres = [x['genre'] for x in movie_data]

In [ ]:
# Create and train a model instance
agglom = AgglomerativeClustering(n_clusters=12, affinity='cosine', linkage='complete')
agglom.fit(movie_vectors)

# Print the resulting cluster labels
print(agglom.labels_)

In [ ]:
# Create and train a model instance, and also get training predictions
agglom = AgglomerativeClustering(n_clusters=12, affinity='euclidean', linkage='complete')
agglom_labels = agglom.fit_predict(movie_vectors)

# Print the resulting cluster labels
print(agglom_labels)

In [ ]:
def print_clusters(cluster_labels):
    n_clusters = max(cluster_labels) + 1
    
    # Group cluster indices
    cluster_indices = [[] for i in range(n_clusters)]
    for i, label in enumerate(cluster_labels):
        cluster_indices[label].append(i)
    
    # Print clusters
    for cluster_id in range(n_clusters):
        print("Cluster {}".format(cluster_id))
        
        for movie_id in cluster_indices[cluster_id]:
            struct = movie_data[movie_id]
            struct_genre = struct["genre"]
            print("  {} - {}".format(struct_genre, struct["genre"]))

In [ ]:
agglom = AgglomerativeClustering(n_clusters=8, affinity='cosine', linkage='complete')
agglom_labels = agglom.fit_predict(movie_vectors)

print_clusters(agglom_labels)

In [ ]:
# Code modified from "Plot Hierarchical Clustering Dendrogram"
def plot_dendrogram(model, color_level=None, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # Create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    if color_level is not None:
        color_threshold=sorted(linkage_matrix[:,2])[-color_level+1]
        dendrogram(linkage_matrix, color_threshold=color_threshold, **kwargs)
    else:
        # Plot the corresponding dendrogram
        dendrogram(linkage_matrix, **kwargs)

In [ ]:
plt.figure(figsize=(15,4)) # Specifies the figure size of the plot.

agglom_model = AgglomerativeClustering(n_clusters=None, distance_threshold=0, affinity='euclidean', linkage='complete', )
agglom_model.fit(movie_vectors)

# color_level 
plot_dendrogram(agglom_model, color_level=3)

_ = plt.title('Agglomerative Clustering Dendrogram')

In [ ]:
plt.figure(figsize=(15,4))
plot_dendrogram(agglom_model, color_level=3, leaf_rotation=90, leaf_font_size=12, labels=movie_genres)
_ = plt.title('Dendrogram with Genre Labels')